In [1]:
!pip install streamlit faiss-cpu sentence-transformers pandas requests pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

In [2]:
import requests
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Amadeus API credentials
API_KEY = "m9kXlBXW2EhnCeRnQavnBbizXcsnkp7Z"
API_SECRET = "wKjSqYBFAht0MJBD"

# Function to get API token
def get_amadeus_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": API_KEY,
        "client_secret": API_SECRET
    }
    response = requests.post(url, headers=headers, data=data)
    return response.json()["access_token"]

# Function to fetch flight prices
def fetch_flights(origin, destination, departure_date):
    token = get_amadeus_token()
    url = f"https://test.api.amadeus.com/v2/shopping/flight-offers"
    headers = {"Authorization": f"Bearer {token}"}

    params = {
        "originLocationCode": origin,
        "destinationLocationCode": destination,
        "departureDate": departure_date,
        "adults": 1
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        flights = []
        for flight in data.get("data", [])[:5]:  # Retrieve top 5 flights
            flights.append({
                "Airline": flight["validatingAirlineCodes"][0],
                "Price": flight["price"]["total"],
                "Departure": flight["itineraries"][0]["segments"][0]["departure"]["at"],
                "Arrival": flight["itineraries"][0]["segments"][-1]["arrival"]["at"],
                "Duration": flight["itineraries"][0]["duration"],
                "Stops": len(flight["itineraries"][0]["segments"]),
                "Route": f"{origin} to {destination}"
            })
        return flights
    else:
        return []

# Example: Fetch flight prices from JFK to LAX
flights_data = fetch_flights("JFK", "LAX", "2025-02-20")

# Convert to DataFrame
df = pd.DataFrame(flights_data)

# Save dataset for FAISS retrieval
df.to_csv("flights_data.csv", index=False)
print("✅ Flight data saved!")


✅ Flight data saved!


In [3]:
# Load the dataset
df = pd.read_csv("flights_data.csv")

# Select relevant text fields
df["text"] = (
    "Flight from " + df["Route"] +
    ", Airline: " + df["Airline"] +
    ", Price: $" + df["Price"].astype(str) +
    ", Duration: " + df["Duration"] +
    ", Stops: " + df["Stops"].astype(str)
)

# Load a sentence transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings
embeddings = model.encode(df["text"].tolist(), convert_to_numpy=True)

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Save FAISS index
faiss.write_index(index, "flight_faiss_index.bin")

print("✅ FAISS index created and saved!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ FAISS index created and saved!


In [4]:
def retrieve_flights(query, top_k=3):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    results = df.iloc[indices[0]]
    return results


In [8]:
%%writefile app.py

import streamlit as st
import pandas as pd
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load dataset
df = pd.read_csv("flights_data.csv")

# Combine relevant fields into text
df["text"] = (
    "Flight from " + df["Route"] +
    ", Airline: " + df["Airline"] +
    ", Price: $" + df["Price"].astype(str) +
    ", Duration: " + df["Duration"] +
    ", Stops: " + df["Stops"].astype(str)
)

# Load sentence transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Load FAISS index
index = faiss.read_index("flight_faiss_index.bin")

# ✅ Fix: Define the function inside `app.py`
def retrieve_flights(query, top_k=3):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    results = df.iloc[indices[0]]
    return results

# Streamlit UI
st.title("✈ AI-Powered Flight Search System")

query = st.text_input("🔍 Enter your flight preference (e.g., 'Cheapest direct flight from JFK to LAX'): ")

if query:
    results = retrieve_flights(query)
    st.subheader("💰 Matching Flights")
    for _, row in results.iterrows():
        st.write(f"🛫 **Airline:** {row['Airline']}")
        st.write(f"💰 **Price:** ${row['Price']}")
        st.write(f"📍 **From:** {row['Route']}")
        st.write(f"🕒 **Duration:** {row['Duration']}")
        st.write(f"🚏 **Stops:** {row['Stops']}")
        st.write("---")



Overwriting app.py


In [13]:
!pip install pyngrok
from pyngrok import ngrok

In [14]:
# Run Streamlit app
!streamlit run app.py &>/content/logs.txt &
# Set up ngrok
!ngrok authtoken 2suOuuxs3zjz3pWQWDl9dQZPTLR_5FKMVwXpnfPJDKgESoGpK

public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://9b07-34-125-203-39.ngrok-free.app" -> "http://localhost:8501"
